In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install imbalanced-learn

In [6]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from  xgboost import XGBClassifier
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [8]:
allHyperTest = pd.read_csv("allhyperTestEDIT.CSV")
allHyperTrain = pd.read_csv("allhyperTrainEDIT.CSV")
allHypoTest = pd.read_csv("allhypoTEST.csv")
allHypoTrain = pd.read_csv("allhypoDATA.CSV")

display(allHypoTest.head(10))
display(allHypoTrain.dtypes)

age sex on_thyroxine query_on_thyroxine on_antithyroid_medication sick  \
0   35   F            f                  f                         f    f   
1   63   M            f                  f                         f    f   
2   25   F            f                  f                         f    f   
3   53   F            f                  f                         f    f   
4   92   F            f                  f                         f    f   
5   67   M            f                  f                         f    f   
6   60   F            f                  f                         f    f   
7   60   F            f                  f                         f    f   
8   48   F            f                  f                         f    f   
9   27   F            f                  f                         f    f   

  pregnant thyroid_surgery I131_treatment query_hypothyroid  ...  TT4  \
0        f               f              f                 f  ...    ?   
1        f               f              f                 f  ...  108   
2        f               f              f                 f  ...   61   
3        f               f              f                 t  ...  145   
4        f               f              f                 f  ...  120   
5        f               f              f                 t  ...   84   
6        f               f              f                 f  ...  117   
7        f               f              f                 f  ...   65   
8        f               f              f                 f  ...  112   
9        f               f              f                 f  ...   94   

  T4U_measured   T4U FTI_measured  FTI TBG_measured TBG referral_source  \
0            f     ?            f    ?            f   ?           other   
1            t  0.96            t  113            f   ?             SVI   
2            t  0.82            t   75            f   ?            SVHD   
3            t  1.03            t  141            f   ?           other   
4            t  0.84            t  143            f   ?             SVI   
5            t  0.83            t  101            f   ?           other   
6            t  1.31            t   90            f   ?           other   
7            t  0.99            t   66            f   ?             SVI   
8            t  0.92            t  121            f   ?           other   
9            t  0.89            t  106            f   ?             SVI   

                    Target    ID  
0                 negative   219  
1                 negative  2059  
2                 negative   399  
3                 negative  1911  
4                 negative   487  
5                 negative  1234  
6                 negative  1113  
7  compensated_hypothyroid  1344  
8                 negative  2758  
9                 negative  3230  

[10 rows x 31 columns]

age                          object
sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
sick                         object
pregnant                     object
thyroid_surgery              object
I131_treatment               object
query_hypothyroid            object
query_hyperthyroid           object
lithium                      object
goitre                       object
tumor                        object
hypopituitary                object
psych                        object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                         

In [9]:
def handleDuplicated(df):
    if df["ID"].duplicated().sum() == 0 :
        print("There aren't duplicates")
    elif (df["ID"].duplicated().sum()) < len(df) / 100:
        df["ID"].drop_duplicates(keep="first", inplace=True)
        print("duplicates were less than the 1% of all the data, they have been dropped")
    else:
        index_duplicated = df["ID"].duplicated().index
        print("duplicates are more than the 1% of all the data, they have been preserved")
        print(index_duplicated)

handleDuplicated(allHyperTest)
handleDuplicated(allHyperTrain)
handleDuplicated(allHypoTest)
handleDuplicated(allHypoTrain)

There aren't duplicates
There aren't duplicates
There aren't duplicates
There aren't duplicates


In [10]:
del allHyperTest["ID"]
del allHyperTrain["ID"]
del allHypoTest["ID"]
del allHypoTrain["ID"]

In [11]:
def notCorrect_TargetFilter(df,correct_Target,target):
    df = df[df.Target.isin(correct_Target)]
    df.replace(correct_Target,target,inplace = True)
    return df
    
allHyperTest = notCorrect_TargetFilter(allHyperTest,["hyperthyroid","T3_toxic","goitre","secondary_toxic"],"hyperthyroid")
allHyperTrain = notCorrect_TargetFilter(allHyperTrain,["hyperthyroid","T3_toxic","goitre","secondary_toxic"],"hyperthyroid")
allHypoTest = notCorrect_TargetFilter(allHypoTest,["hypothyroid", "primary_hypothyroid", "compensated_hypothyroid", "secondary_hypothyroid"],"hypothyroid")
allHypoTrain = notCorrect_TargetFilter(allHypoTrain,["hypothyroid", "primary_hypothyroid", "compensated_hypothyroid", "secondary_hypothyroid"],"hypothyroid")

C:\Users\DELL\AppData\Local\Temp\ipykernel_356\1772405005.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(correct_Target,target,inplace = True)


In [12]:
allDataset = pd.concat([allHyperTest,allHyperTrain,allHypoTest,allHypoTrain], ignore_index = True)
display(allDataset.shape)

(393, 30)

In [13]:
allDataset

age sex on_thyroxine query_on_thyroxine on_antithyroid_medication sick  \
0    79   M            f                  f                         f    f   
1    50   F            f                  f                         t    f   
2    63   ?            f                  f                         f    f   
3    58   F            f                  f                         f    f   
4    39   F            f                  f                         f    f   
..   ..  ..          ...                ...                       ...  ...   
388  60   F            f                  f                         f    f   
389  67   F            f                  f                         f    f   
390  46   F            f                  f                         f    f   
391  24   F            f                  f                         f    f   
392  25   F            f                  f                         f    f   

    pregnant thyroid_surgery I131_treatment query_hypothyroid  ...  \
0          f               f              f                 f  ...   
1          f               f              f                 f  ...   
2          f               f              f                 f  ...   
3          f               f              f                 t  ...   
4          f               f              f                 f  ...   
..       ...             ...            ...               ...  ...   
388        f               f              f                 t  ...   
389        f               f              f                 f  ...   
390        f               f              f                 f  ...   
391        f               f              f                 f  ...   
392        f               f              f                 f  ...   

    TT4_measured  TT4 T4U_measured   T4U FTI_measured  FTI TBG_measured TBG  \
0              t  151            t  0.74            t  203            f   ?   
1              t  131            t  1.02            t  129            f   ?   
2              t  162            t  0.81            t  200            f   ?   
3              t  256            t   0.9            t  283            f   ?   
4              t  209            t  1.06            t  197            f   ?   
..           ...  ...          ...   ...          ...  ...          ...  ..   
388            t   45            t  0.97            t   46            f   ?   
389            t   48            t  0.98            t   49            f   ?   
390            t   48            t  0.89            t   54            f   ?   
391            t  110            t  1.17            t   94            f   ?   
392            t   21            t   1.1            t   19            f   ?   

    referral_source        Target  
0             other  hyperthyroid  
1             other  hyperthyroid  
2             other  hyperthyroid  
3             other  hyperthyroid  
4             other  hyperthyroid  
..              ...           ...  
388           other   hypothyroid  
389             SVI   hypothyroid  
390           other   hypothyroid  
391           other   hypothyroid  
392           other   hypothyroid  

[393 rows x 30 columns]

In [14]:
thyroid0387 = pd.read_csv("thyroid0387EDIT.CSV")
display(thyroid0387.head(10))
display(thyroid0387.dtypes)

age sex on_thyroxine query_on_thyroxine on_antithyroid_medication sick  \
0   29   F            f                  f                         f    f   
1   29   F            f                  f                         f    f   
2   41   F            f                  f                         f    f   
3   36   F            f                  f                         f    f   
4   32   F            f                  f                         f    f   
5   60   F            f                  f                         f    f   
6   77   F            f                  f                         f    f   
7   28   F            f                  f                         f    f   
8   28   F            f                  f                         f    f   
9   28   F            f                  f                         f    f   

  pregnant thyroid_surgery I131_treatment query_hypothyroid  ...  TT4  \
0        f               f              f                 t  ...    ?   
1        f               f              f                 f  ...  128   
2        f               f              f                 f  ...    ?   
3        f               f              f                 f  ...    ?   
4        f               f              f                 f  ...    ?   
5        f               f              f                 f  ...    ?   
6        f               f              f                 f  ...    ?   
7        f               f              f                 f  ...  116   
8        f               f              f                 f  ...   76   
9        f               f              f                 f  ...   83   

  T4U_measured T4U FTI_measured FTI TBG_measured TBG referral_source Target  \
0            f   ?            f   ?            f   ?           other      -   
1            f   ?            f   ?            f   ?           other      -   
2            f   ?            f   ?            t  11           other      -   
3            f   ?            f   ?            t  26           other      -   
4            f   ?            f   ?            t  36           other      S   
5            f   ?            f   ?            t  26           other      -   
6            f   ?            f   ?            t  21           other      -   
7            f   ?            f   ?            f   ?             SVI      -   
8            f   ?            f   ?            f   ?           other      -   
9            f   ?            f   ?            f   ?           other      -   

          ID  
0  840801013  
1  840801014  
2  840801042  
3  840803046  
4  840803047  
5  840803048  
6  840803068  
7  840807019  
8  840808060  
9  840808073  

[10 rows x 31 columns]

age                           int64
sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
sick                         object
pregnant                     object
thyroid_surgery              object
I131_treatment               object
query_hypothyroid            object
query_hyperthyroid           object
lithium                      object
goitre                       object
tumor                        object
hypopituitary                object
psych                        object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                         

In [15]:
handleDuplicated(thyroid0387)

There aren't duplicates


In [16]:
del thyroid0387["ID"]

In [17]:
thyroid0387['sex'] = thyroid0387['sex'].map({'F': 1, 'M': 0})

thyroid0387.replace(['A', 'B', 'C', 'D'], "hyperthyroid", inplace=True)
thyroid0387.replace(['E', 'F', 'G', 'H'], "hypothyroid", inplace=True)

for value in set(thyroid0387['Target']):
    if value != 'hypothyroid' and value != 'hyperthyroid':
        thyroid0387.replace(value, 'negative', inplace=True)


In [18]:
thyroid0387

age  sex on_thyroxine query_on_thyroxine on_antithyroid_medication sick  \
0      29  1.0            f                  f                         f    f   
1      29  1.0            f                  f                         f    f   
2      41  1.0            f                  f                         f    f   
3      36  1.0            f                  f                         f    f   
4      32  1.0            f                  f                         f    f   
...   ...  ...          ...                ...                       ...  ...   
9167   56  0.0            f                  f                         f    f   
9168   22  0.0            f                  f                         f    f   
9169   69  0.0            f                  f                         f    f   
9170   47  1.0            f                  f                         f    f   
9171   31  0.0            f                  f                         f    f   

     pregnant thyroid_surgery I131_treatment query_hypothyroid  ...  \
0           f               f              f                 t  ...   
1           f               f              f                 f  ...   
2           f               f              f                 f  ...   
3           f               f              f                 f  ...   
4           f               f              f                 f  ...   
...       ...             ...            ...               ...  ...   
9167        f               f              f                 f  ...   
9168        f               f              f                 f  ...   
9169        f               f              f                 f  ...   
9170        f               f              f                 f  ...   
9171        f               f              f                 t  ...   

     TT4_measured  TT4 T4U_measured   T4U FTI_measured FTI TBG_measured TBG  \
0               f    ?            f     ?            f   ?            f   ?   
1               t  128            f     ?            f   ?            f   ?   
2               f    ?            f     ?            f   ?            t  11   
3               f    ?            f     ?            f   ?            t  26   
4               f    ?            f     ?            f   ?            t  36   
...           ...  ...          ...   ...          ...  ..          ...  ..   
9167            t   64            t  0.83            t  77            f   ?   
9168            t   91            t  0.92            t  99            f   ?   
9169            t  113            t  1.27            t  89            f   ?   
9170            t   75            t  0.85            t  88            f   ?   
9171            t   66            t  1.02            t  65            f   ?   

     referral_source    Target  
0              other  negative  
1              other  negative  
2              other  negative  
3              other  negative  
4              other  negative  
...              ...       ...  
9167             SVI  negative  
9168             SVI  negative  
9169             SVI  negative  
9170           other  negative  
9171           other  negative  

[9172 rows x 30 columns]

In [19]:
hypothyroid = pd.read_csv("hypothyroid.csv")
display(hypothyroid.shape)
display(hypothyroid.head(10))
display(hypothyroid.dtypes)

(3163, 26)

Unnamed: 0 Age Sex on_thyroxine query_on_thyroxine  \
0  hypothyroid  72   M            f                  f   
1  hypothyroid  15   F            t                  f   
2  hypothyroid  24   M            f                  f   
3  hypothyroid  24   F            f                  f   
4  hypothyroid  77   M            f                  f   
5  hypothyroid  85   F            f                  f   
6  hypothyroid  64   F            f                  f   
7  hypothyroid  72   F            f                  f   
8  hypothyroid  20   F            f                  f   
9  hypothyroid  42   F            f                  f   

  on_antithyroid_medication thyroid_surgery query_hypothyroid  \
0                         f               f                 f   
1                         f               f                 f   
2                         f               f                 f   
3                         f               f                 f   
4                         f               f                 f   
5                         f               f                 t   
6                         f               t                 f   
7                         f               f                 f   
8                         f               f                 t   
9                         f               f                 f   

  query_hyperthyroid pregnant  ... T3_measured   T3 TT4_measured  TT4  \
0                  f        f  ...           y  0.6            y   15   
1                  f        f  ...           y  1.7            y   19   
2                  f        f  ...           y  0.2            y    4   
3                  f        f  ...           y  0.4            y    6   
4                  f        f  ...           y  1.2            y   57   
5                  f        f  ...           y  1.1            y   27   
6                  f        f  ...           y  1.3            y   54   
7                  f        f  ...           y  1.9            y   34   
8                  f        f  ...           n    ?            y   39   
9                  f        f  ...           n    ?            y  7.6   

  T4U_measured   T4U FTI_measured  FTI TBG_measured TBG  
0            y  1.48            y   10            n   ?  
1            y  1.13            y   17            n   ?  
2            y     1            y    0            n   ?  
3            y  1.04            y    6            n   ?  
4            y  1.28            y   44            n   ?  
5            y  1.19            y   23            n   ?  
6            y  0.86            y   63            n   ?  
7            y  1.05            y   32            n   ?  
8            y  1.21            y   32            n   ?  
9            y  1.02            y  7.5            n   ?  

[10 rows x 26 columns]

Unnamed: 0                   object
Age                          object
Sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
thyroid_surgery              object
query_hypothyroid            object
query_hyperthyroid           object
pregnant                     object
sick                         object
tumor                        object
lithium                      object
goitre                       object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                          object
dtype: object

In [20]:
hypothyroid = hypothyroid.rename(columns={hypothyroid.columns[0]:"Target",hypothyroid.columns[1]:"age",hypothyroid.columns[2]:"sex" })
hypothyroid = hypothyroid[hypothyroid.Target.isin(['hypothyroid'])]

In [21]:
hypothyroid

Target age sex on_thyroxine query_on_thyroxine  \
0    hypothyroid  72   M            f                  f   
1    hypothyroid  15   F            t                  f   
2    hypothyroid  24   M            f                  f   
3    hypothyroid  24   F            f                  f   
4    hypothyroid  77   M            f                  f   
..           ...  ..  ..          ...                ...   
146  hypothyroid  78   F            f                  f   
147  hypothyroid  69   F            f                  f   
148  hypothyroid   ?   F            f                  f   
149  hypothyroid  30   F            t                  f   
150  hypothyroid  34   F            f                  f   

    on_antithyroid_medication thyroid_surgery query_hypothyroid  \
0                           f               f                 f   
1                           f               f                 f   
2                           f               f                 f   
3                           f               f                 f   
4                           f               f                 f   
..                        ...             ...               ...   
146                         f               f                 f   
147                         f               f                 t   
148                         f               f                 f   
149                         f               f                 f   
150                         f               f                 f   

    query_hyperthyroid pregnant  ... T3_measured   T3 TT4_measured  TT4  \
0                    f        f  ...           y  0.6            y   15   
1                    f        f  ...           y  1.7            y   19   
2                    f        f  ...           y  0.2            y    4   
3                    f        f  ...           y  0.4            y    6   
4                    f        f  ...           y  1.2            y   57   
..                 ...      ...  ...         ...  ...          ...  ...   
146                  f        f  ...           y  0.2            y   17   
147                  f        f  ...           y  1.5            y   50   
148                  f        f  ...           y  0.7            y   14   
149                  f        f  ...           y    3            y  109   
150                  f        f  ...           y  0.2            y  8.6   

    T4U_measured   T4U FTI_measured  FTI TBG_measured TBG  
0              y  1.48            y   10            n   ?  
1              y  1.13            y   17            n   ?  
2              y     1            y    0            n   ?  
3              y  1.04            y    6            n   ?  
4              y  1.28            y   44            n   ?  
..           ...   ...          ...  ...          ...  ..  
146            y  1.01            y   16            n   ?  
147            y  0.83            y   60            n   ?  
148            y  1.13            y   12            n   ?  
149            y  1.79            y   61            n   ?  
150            y  0.99            y  8.7            n   ?  

[151 rows x 26 columns]

In [22]:
sick_euthyroid = pd.read_csv("sick-euthyroid.CSV")
display(sick_euthyroid.shape)
display(sick_euthyroid.head(10))
display(sick_euthyroid.dtypes)

(3163, 26)

Target age sex on_thyroxine query_on_thyroxine  \
0  sick-euthyroid  72   M            f                  f   
1  sick-euthyroid  45   F            f                  f   
2  sick-euthyroid  64   F            f                  f   
3  sick-euthyroid  56   M            f                  f   
4  sick-euthyroid  78   F            t                  f   
5  sick-euthyroid  80   M            f                  f   
6  sick-euthyroid  74   F            f                  f   
7  sick-euthyroid   ?   F            f                  f   
8  sick-euthyroid  42   F            f                  f   
9  sick-euthyroid  89   M            f                  f   

  on_antithyroid_medication thyroid_surgery query_hypothyroid  \
0                         f               f                 f   
1                         f               f                 f   
2                         f               f                 f   
3                         f               f                 f   
4                         f               f                 t   
5                         f               f                 f   
6                         f               f                 f   
7                         f               f                 f   
8                         f               f                 f   
9                         f               f                 f   

  query_hyperthyroid pregnant  ... T3_measured    T3 TT4_measured  TT4  \
0                  f        f  ...           y     1            y   83   
1                  f        f  ...           y     1            y   82   
2                  f        f  ...           y     1            y  101   
3                  f        f  ...           y  0.80            y   76   
4                  f        f  ...           y  0.30            y   87   
5                  f        f  ...           y  0.80            y  105   
6                  f        f  ...           y  0.70            y   98   
7                  f        f  ...           y  1.10            y  121   
8                  f        f  ...           y  1.10            y   93   
9                  f        f  ...           y  0.80            y  111   

  T4U_measured   T4U FTI_measured  FTI TBG_measured TBG  
0            y  0.95            y   87            n   ?  
1            y  0.73            y  112            n   ?  
2            y  0.82            y  123            n   ?  
3            y  0.77            y   99            n   ?  
4            y  0.95            y   91            n   ?  
5            y  0.88            y  120            n   ?  
6            y  0.81            y  121            n   ?  
7            y  1.11            y  109            n   ?  
8            y  0.73            y  127            n   ?  
9            y  0.68            y  165            n   ?  

[10 rows x 26 columns]

Target                       object
age                          object
sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
thyroid_surgery              object
query_hypothyroid            object
query_hyperthyroid           object
pregnant                     object
sick                         object
tumor                        object
lithium                      object
goitre                       object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                          object
dtype: object

In [23]:
sick_euthyroid = sick_euthyroid[sick_euthyroid.Target.isin(['negative'])]
display(sick_euthyroid.shape)

(2870, 26)

In [24]:
sick_euthyroid

Target age sex on_thyroxine query_on_thyroxine  \
293   negative  20   F            f                  f   
294   negative  29   F            f                  f   
295   negative  66   F            f                  f   
296   negative  62   M            f                  f   
297   negative  72   F            f                  f   
...        ...  ..  ..          ...                ...   
3158  negative  40   F            f                  f   
3159  negative  69   F            f                  f   
3160  negative  58   F            f                  f   
3161  negative  29   F            f                  f   
3162  negative  56   F            t                  f   

     on_antithyroid_medication thyroid_surgery query_hypothyroid  \
293                          f               f                 f   
294                          f               f                 f   
295                          f               f                 f   
296                          f               f                 f   
297                          f               f                 f   
...                        ...             ...               ...   
3158                         f               f                 f   
3159                         f               f                 f   
3160                         f               f                 f   
3161                         f               f                 f   
3162                         f               f                 f   

     query_hyperthyroid pregnant  ... T3_measured    T3 TT4_measured  TT4  \
293                   f        f  ...           y  1.80            y   68   
294                   f        f  ...           y  2.50            y   83   
295                   t        f  ...           y  1.70            y   76   
296                   f        f  ...           y  2.20            y  103   
297                   f        f  ...           y  1.50            y   66   
...                 ...      ...  ...         ...   ...          ...  ...   
3158                  f        f  ...           y  1.20            y   76   
3159                  f        f  ...           y  1.80            y  126   
3160                  f        f  ...           y  1.70            y   86   
3161                  f        f  ...           y  1.80            y   99   
3162                  f        f  ...           y  1.80            y  139   

     T4U_measured   T4U FTI_measured  FTI TBG_measured TBG  
293             y  0.99            y   68            n   ?  
294             y  0.93            y   89            n   ?  
295             y  0.83            y   92            n   ?  
296             y  0.99            y  104            n   ?  
297             y  0.97            y   69            n   ?  
...           ...   ...          ...  ...          ...  ..  
3158            y  0.90            y   84            n   ?  
3159            y  1.02            y  124            n   ?  
3160            y  0.91            y   95            n   ?  
3161            y  1.01            y   98            n   ?  
3162            y  0.97            y  143            n   ?  

[2870 rows x 26 columns]

In [25]:
ann_train = pd.read_csv("ann-train.CSV")
ann_test = pd.read_csv("ann-test.CSV")
display(ann_test.head(10))
display(ann_test.dtypes)

age  sex  on_thyroxine  query_on_thyroxine  on_antithyroid_medication  \
0  0.29    0             0                   0                          0   
1  0.32    0             0                   0                          0   
2  0.35    0             0                   0                          0   
3  0.21    0             0                   0                          0   
4  0.22    0             0                   0                          0   
5  0.22    0             0                   0                          0   
6  0.39    0             0                   0                          0   
7  0.77    1             0                   0                          0   
8  0.23    0             0                   0                          0   
9  0.23    0             0                   0                          0   

   sick  pregnant  thyroid_surgery  I131_treatment  query_hypothyroid  ...  \
0     0         0                0               0                  0  ...   
1     0         0                0               0                  0  ...   
2     0         0                0               0                  0  ...   
3     0         0                0               0                  0  ...   
4     1         0                0               0                  0  ...   
5     0         0                0               0                  0  ...   
6     0         0                0               0                  0  ...   
7     0         0                0               0                  0  ...   
8     0         0                0               0                  0  ...   
9     0         0                0               0                  0  ...   

   goitre  tumor  hypopituitary  psych      TSH     T3    TT4    T4U      FTI  \
0       0      0              0      0  0.00610  0.028  0.111  0.131  0.08500   
1       0      0              0      0  0.00130  0.019  0.084  0.078  0.10700   
2       0      0              0      0  0.00000  0.031  0.239  0.100  0.23900   
3       0      0              0      0  0.00100  0.018  0.087  0.088  0.09900   
4       0      0              0      0  0.00040  0.022  0.134  0.135  0.09900   
5       0      0              0      0  0.00160  0.020  0.123  0.113  0.10900   
6       0      0              0      0  0.00160  0.036  0.133  0.144  0.09300   
7       0      0              0      0  0.00081  0.020  0.080  0.096  0.08316   
8       0      0              0      0  0.00025  0.014  0.113  0.096  0.11746   
9       0      0              0      0  0.00260  0.011  0.104  0.104  0.09900   

   Target  
0       2  
1       3  
2       3  
3       3  
4       3  
5       3  
6       3  
7       3  
8       3  
9       3  

[10 rows x 22 columns]

age                          float64
sex                            int64
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment                 int64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                  int64
psych                          int64
TSH                          float64
T3                           float64
TT4                          float64
T4U                          float64
FTI                          float64
Target                         int64
dtype: object

In [26]:
target1 = pd.Series(ann_test[ann_test.columns[-1]].values)
display(target1.value_counts())
target2 = pd.Series(ann_train[ann_train.columns[-1]].values)
display(target2.value_counts())

3    3178
2     177
1      73
Name: count, dtype: int64

3    3488
2     191
1      93
Name: count, dtype: int64

In [27]:
#3 is referring to the 'negative' class
#2 is referring to the 'hypothyroid' class
# 1 is referring to the 'hyperthyroid' class

In [28]:
print("Sex thyroid0387 1=F,0=M:")
sex_series1 = pd.Series(thyroid0387[thyroid0387.columns[1]].values)
display(sex_series1.value_counts())
print("Sick-euthyroid:")
sex_series2 = pd.Series(sick_euthyroid[sick_euthyroid.columns[2]].values)
display(sex_series2.value_counts())

Sex thyroid0387 1=F,0=M:


1.0    6073
0.0    2792
Name: count, dtype: int64

Sick-euthyroid:


F    2003
M     800
?      67
Name: count, dtype: int64

In [29]:
sex1 = pd.Series(ann_test[ann_test.columns[1]].values)
display(sex1.value_counts())
sex2 = pd.Series(ann_train[ann_train.columns[1]].values)
display(sex2.value_counts())

0    2380
1    1048
Name: count, dtype: int64

0    2629
1    1143
Name: count, dtype: int64

In [30]:
for column in ann_train.columns:
    listOfValues=set(ann_train[column])
    print(column,": ",listOfValues)

age :  {0.73, 0.24, 0.47, 0.69, 0.48, 0.76, 0.5, 0.25, 0.26, 0.51, 0.75, 0.23, 0.22, 0.54, 0.79, 0.21, 0.63, 0.38, 0.13, 0.88, 0.72, 0.42, 0.18, 0.43, 0.44, 0.19, 0.2, 0.45, 0.46, 0.41, 0.94, 0.15, 0.16, 0.17, 0.55, 0.8, 0.39, 0.89, 0.14, 0.515, 0.57, 0.82, 0.64, 0.85, 0.67, 0.59, 0.58, 0.84, 0.92, 0.68, 0.35, 0.6, 0.93, 0.11, 0.77, 0.27, 0.52, 0.91, 0.61, 0.86, 0.36, 0.7, 0.1, 0.08, 0.56, 0.81, 0.65, 0.4, 0.9, 0.49, 0.74, 0.83, 0.34, 0.29, 0.3, 0.31, 0.32, 0.33, 0.07, 0.06, 0.05, 0.04, 0.53, 0.78, 0.28, 0.62, 0.37, 0.12, 0.87, 0.71, 0.02, 0.01, 0.66}
sex :  {0, 1}
on_thyroxine :  {0, 1}
query_on_thyroxine :  {0, 1}
on_antithyroid_medication :  {0, 1}
sick :  {0, 1}
pregnant :  {0, 1}
thyroid_surgery :  {0, 1}
I131_treatment :  {0, 1}
query_hypothyroid :  {0, 1}
query_hyperthyroid :  {0, 1}
lithium :  {0, 1}
goitre :  {0, 1}
tumor :  {0, 1}
hypopituitary :  {0, 1}
psych :  {0, 1}
TSH :  {0.139, 0.0, 0.472, 0.236, 0.478, 0.00061, 0.53, 0.00044, 0.0061, 0.00027, 0.0041, 0.00088, 0.0001, 

In [31]:
ann = pd.concat([ann_train,ann_test], ignore_index = True)
ann['sex'] = ann['sex'].map({0:'F',1:'M'})
ann['Target'] = ann['Target'].map({3:'negative',2:'hypothyroid',1:'hyperthyroid'})

continuos_attributes = ['age','TSH','T3','TT4','T4U','FTI']
for attribute in continuos_attributes:
    ann[attribute] = ann[attribute] * 100

def fillNewAttributes(row,attribute):
    if row[attribute] > 0:
        return 'y'
    else:
        return 'n'

ann['TSH_measured'] = ann.apply(lambda row: fillNewAttributes(row,'TSH'), axis=1)
ann['T3_measured'] = ann.apply(lambda row: fillNewAttributes(row,'T3'), axis=1)
ann['TT4_measured'] = ann.apply(lambda row: fillNewAttributes(row,'TT4'), axis=1)
ann['T4U_measured'] = ann.apply(lambda row: fillNewAttributes(row,'T4U'), axis=1)
ann['FTI_measured'] = ann.apply(lambda row: fillNewAttributes(row,'FTI'), axis=1)
display(ann.dtypes)

age                          float64
sex                           object
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment                 int64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                  int64
psych                          int64
TSH                          float64
T3                           float64
TT4                          float64
T4U                          float64
FTI                          float64
Target                        object
TSH_measured                  object
T3_measured                   object
TT4_measured                  object
T4U_measured                  object
FTI_measured                  object
d

In [32]:
data = pd.concat([allDataset,thyroid0387,hypothyroid,sick_euthyroid,ann], ignore_index = True)
display(data.shape)
display(data.dtypes)

(19786, 30)

age                          object
sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
sick                         object
pregnant                     object
thyroid_surgery              object
I131_treatment               object
query_hypothyroid            object
query_hyperthyroid           object
lithium                      object
goitre                       object
tumor                        object
hypopituitary                object
psych                        object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                         

In [33]:
data

age sex on_thyroxine query_on_thyroxine on_antithyroid_medication  \
0        79   M            f                  f                         f   
1        50   F            f                  f                         t   
2        63   ?            f                  f                         f   
3        58   F            f                  f                         f   
4        39   F            f                  f                         f   
...     ...  ..          ...                ...                       ...   
19781  59.0   F            0                  0                         0   
19782  51.0   F            0                  0                         0   
19783  51.0   F            0                  0                         0   
19784  35.0   M            0                  0                         0   
19785  73.0   F            0                  0                         0   

      sick pregnant thyroid_surgery I131_treatment query_hypothyroid  ...  \
0        f        f               f              f                 f  ...   
1        f        f               f              f                 f  ...   
2        f        f               f              f                 f  ...   
3        f        f               f              f                 t  ...   
4        f        f               f              f                 f  ...   
...    ...      ...             ...            ...               ...  ...   
19781    0        0               0              0                 0  ...   
19782    0        0               0              0                 0  ...   
19783    0        0               0              0                 0  ...   
19784    0        0               0              0                 0  ...   
19785    0        0               0              0                 0  ...   

      TT4_measured  TT4 T4U_measured   T4U FTI_measured   FTI TBG_measured  \
0                t  151            t  0.74            t   203            f   
1                t  131            t  1.02            t   129            f   
2                t  162            t  0.81            t   200            f   
3                t  256            t   0.9            t   283            f   
4                t  209            t  1.06            t   197            f   
...            ...  ...          ...   ...          ...   ...          ...   
19781            y  7.9            y   9.9            y   8.0          NaN   
19782            y  0.5            y   8.9            y  0.55          NaN   
19783            y  9.0            y   6.7            y  13.4          NaN   
19784            y  9.0            y   8.9            y  10.1          NaN   
19785            y  8.1            y   9.0            y   9.0          NaN   

       TBG referral_source        Target  
0        ?           other  hyperthyroid  
1        ?           other  hyperthyroid  
2        ?           other  hyperthyroid  
3        ?           other  hyperthyroid  
4        ?           other  hyperthyroid  
...    ...             ...           ...  
19781  NaN             NaN      negative  
19782  NaN             NaN  hyperthyroid  
19783  NaN             NaN      negative  
19784  NaN             NaN      negative  
19785  NaN             NaN      negative  

[19786 rows x 30 columns]

In [34]:
# data preprocessing

In [35]:
for column in data.columns:
    listOfValues=set(data[column])
    print(column,": ",listOfValues)

age :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, '88', 15, 16, 14, 18, 19, 20, 21, 17, 23, 24, 25, 26, 27, 28, '30', 29, 31, 32, 33, 34, '59', 35, 37, 38, 39, '51', 40, 36, 42, '34', 45, 43, 41, 46, '50', 50, 51, 44, 53, 54, 55, 56, 52, 58, 59, 60, 61, '65', 63, '87', 65, 62, '80', 68, 69, '36', 71, 67, 73, '31', 75, 76, 77, 74, 79, 80, 78, '64', 83, '26', 85, 82, 86, 88, 81, 90, 84, 92, 89, 87, '49', 93, 97, 91, 94, 95, 7.000000000000001, '12', '4', 22, '77', '82', '13', '8', '24', '75', '32', '16', 30, '43', '5', '2', '70', '40', '85', '47', 14.000000000000002, '61', '9', '17', '66', '38', '1', '48', '18', '52', '33', '45', 47, '98', '23', '55', 48, '72', '60', 49, '76', 51.5, '67', 52.190000000000005, '97', 55.00000000000001, 56.99999999999999, '58', 56.00000000000001, 57, 57.99999999999999, '86', '21', '73', '46', '42', '19', '92', '10', 64, '11', '44', 66, '29', '7', '27', '68', 70, '54', '41', '25', '83', 72, '79', '35', '63', '90', '81', 28.000000000000004, 28.999999999999996,

In [36]:
data=data.replace({"?":np.NAN})
data.isna().sum()

age                            409
sex                            394
on_thyroxine                     0
query_on_thyroxine               0
on_antithyroid_medication        0
sick                             0
pregnant                         0
thyroid_surgery                  0
I131_treatment                3021
query_hypothyroid                0
query_hyperthyroid               0
lithium                          0
goitre                           0
tumor                            0
hypopituitary                 3021
psych                         3021
TSH_measured                     0
TSH                           1321
T3_measured                      0
T3                            3372
TT4_measured                     0
TT4                            696
T4U_measured                     0
T4U                           1083
FTI_measured                     0
FTI                           1075
TBG_measured                  7200
TBG                          19174
referral_source     

In [37]:
del data['TBG']
del data['referral_source']
del data['TBG_measured']
del data['sex']

In [38]:
data

age on_thyroxine query_on_thyroxine on_antithyroid_medication sick  \
0        79            f                  f                         f    f   
1        50            f                  f                         t    f   
2        63            f                  f                         f    f   
3        58            f                  f                         f    f   
4        39            f                  f                         f    f   
...     ...          ...                ...                       ...  ...   
19781  59.0            0                  0                         0    0   
19782  51.0            0                  0                         0    0   
19783  51.0            0                  0                         0    0   
19784  35.0            0                  0                         0    0   
19785  73.0            0                  0                         0    0   

      pregnant thyroid_surgery I131_treatment query_hypothyroid  \
0            f               f              f                 f   
1            f               f              f                 f   
2            f               f              f                 f   
3            f               f              f                 t   
4            f               f              f                 f   
...        ...             ...            ...               ...   
19781        0               0              0                 0   
19782        0               0              0                 0   
19783        0               0              0                 0   
19784        0               0              0                 0   
19785        0               0              0                 0   

      query_hyperthyroid  ...    TSH T3_measured    T3 TT4_measured  TT4  \
0                      f  ...  0.005           t   3.4            t  151   
1                      t  ...   0.03           t   3.4            t  131   
2                      f  ...    NaN           f   NaN            t  162   
3                      t  ...    0.2           f   NaN            t  256   
4                      t  ...   0.02           t     4            t  209   
...                  ...  ...    ...         ...   ...          ...  ...   
19781                  0  ...   0.25           y  2.08            y  7.9   
19782                  0  ...   10.6           y   0.6            y  0.5   
19783                  0  ...  0.076           y  2.01            y  9.0   
19784                  0  ...   0.28           y  2.01            y  9.0   
19785                  0  ...  0.056           y  2.01            y  8.1   

      T4U_measured   T4U FTI_measured   FTI        Target  
0                t  0.74            t   203  hyperthyroid  
1                t  1.02            t   129  hyperthyroid  
2                t  0.81            t   200  hyperthyroid  
3                t   0.9            t   283  hyperthyroid  
4                t  1.06            t   197  hyperthyroid  
...            ...   ...          ...   ...           ...  
19781            y   9.9            y   8.0      negative  
19782            y   8.9            y  0.55  hyperthyroid  
19783            y   6.7            y  13.4      negative  
19784            y   8.9            y  10.1      negative  
19785            y   9.0            y   9.0      negative  

[19786 rows x 26 columns]

In [39]:
data.dropna(axis = 0, thresh = 20, inplace = True)
data.isna().sum()

age                           380
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
sick                            0
pregnant                        0
thyroid_surgery                 0
I131_treatment               2773
query_hypothyroid               0
query_hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                2773
psych                        2773
TSH_measured                    0
TSH                          1074
T3_measured                     0
T3                           3125
TT4_measured                    0
TT4                           448
T4U_measured                    0
T4U                           835
FTI_measured                    0
FTI                           828
Target                          0
dtype: int64

In [40]:
data

age on_thyroxine query_on_thyroxine on_antithyroid_medication sick  \
0        79            f                  f                         f    f   
1        50            f                  f                         t    f   
2        63            f                  f                         f    f   
3        58            f                  f                         f    f   
4        39            f                  f                         f    f   
...     ...          ...                ...                       ...  ...   
19781  59.0            0                  0                         0    0   
19782  51.0            0                  0                         0    0   
19783  51.0            0                  0                         0    0   
19784  35.0            0                  0                         0    0   
19785  73.0            0                  0                         0    0   

      pregnant thyroid_surgery I131_treatment query_hypothyroid  \
0            f               f              f                 f   
1            f               f              f                 f   
2            f               f              f                 f   
3            f               f              f                 t   
4            f               f              f                 f   
...        ...             ...            ...               ...   
19781        0               0              0                 0   
19782        0               0              0                 0   
19783        0               0              0                 0   
19784        0               0              0                 0   
19785        0               0              0                 0   

      query_hyperthyroid  ...    TSH T3_measured    T3 TT4_measured  TT4  \
0                      f  ...  0.005           t   3.4            t  151   
1                      t  ...   0.03           t   3.4            t  131   
2                      f  ...    NaN           f   NaN            t  162   
3                      t  ...    0.2           f   NaN            t  256   
4                      t  ...   0.02           t     4            t  209   
...                  ...  ...    ...         ...   ...          ...  ...   
19781                  0  ...   0.25           y  2.08            y  7.9   
19782                  0  ...   10.6           y   0.6            y  0.5   
19783                  0  ...  0.076           y  2.01            y  9.0   
19784                  0  ...   0.28           y  2.01            y  9.0   
19785                  0  ...  0.056           y  2.01            y  8.1   

      T4U_measured   T4U FTI_measured   FTI        Target  
0                t  0.74            t   203  hyperthyroid  
1                t  1.02            t   129  hyperthyroid  
2                t  0.81            t   200  hyperthyroid  
3                t   0.9            t   283  hyperthyroid  
4                t  1.06            t   197  hyperthyroid  
...            ...   ...          ...   ...           ...  
19781            y   9.9            y   8.0      negative  
19782            y   8.9            y  0.55  hyperthyroid  
19783            y   6.7            y  13.4      negative  
19784            y   8.9            y  10.1      negative  
19785            y   9.0            y   9.0      negative  

[19538 rows x 26 columns]

In [41]:
data = data.replace({"t":1,"f":0, "y":1, "n":0, "hypothyroid":1, "negative":0,"hyperthyroid":2, "F":1, "M":0})
display(data.dtypes)

age                           object
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment               float64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                float64
psych                        float64
TSH_measured                   int64
TSH                           object
T3_measured                    int64
T3                            object
TT4_measured                   int64
TT4                           object
T4U_measured                   int64
T4U                           object
FTI_measured                   int64
FTI                           object
Target                         int64
dtype: object

In [42]:
cols = data.columns[data.dtypes.eq('object')]
data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')
display(data.dtypes)

age                          float64
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment               float64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                float64
psych                        float64
TSH_measured                   int64
TSH                          float64
T3_measured                    int64
T3                           float64
TT4_measured                   int64
TT4                          float64
T4U_measured                   int64
T4U                          float64
FTI_measured                   int64
FTI                          float64
Target                         int64
dtype: object

In [43]:
corr_values = abs(data[data.columns[0:]].corr()['Target'][:])
corr_values = corr_values.drop('Target')
corr_values = corr_values[corr_values > 0.04]
display(corr_values)

on_thyroxine          0.082979
query_hypothyroid     0.050792
query_hyperthyroid    0.077351
psych                 0.050080
TSH                   0.252200
FTI                   0.046015
Name: Target, dtype: float64

In [44]:
def holdout(dataframe):
    x = dataframe[corr_values.index]
    y = dataframe['Target']
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42) 
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = holdout(data)


In [45]:
classifiers = {
    "XGBClassifier" : XGBClassifier(learning_rate=0.01),
    # 'Random Forest': RandomForestClassifier(max_depth=4,verbose=0),
}

In [46]:

def classification(classifiers, X_train, X_test, y_train, y_test):
    # Initialize res as a DataFrame
    res = pd.DataFrame(columns=["Classifier", "Accuracy", "Precision", "Recall", "FScore"])
    
    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        pr, rc, fs, sup = metrics.precision_recall_fscore_support(y_test, y_pred, average='macro')
        
        # Use loc to append a row to the DataFrame
        res.loc[len(res)] = {
            "Classifier": name,
            "Accuracy": round(metrics.accuracy_score(y_test, y_pred), 4),
            "Precision": round(pr, 4),
            "Recall": round(rc, 4),
            "FScore": round(fs, 4)
        }
        
        print("Confusion matrix for: ", name)
        display(confusion_matrix(y_test, y_pred))
    
    res.set_index("FScore", inplace=True)
    res.sort_values(by="FScore", ascending=False, inplace=True)
    return res

# Call the function with your data
result = classification(classifiers, X_train, X_test, y_train, y_test)
display(result)


Confusion matrix for:  XGBClassifier


array([[5166,   86,   27],
       [  49,  380,    4],
       [  37,    0,  113]], dtype=int64)

Classifier  Accuracy  Precision  Recall
FScore                                            
0.8651  XGBClassifier    0.9654     0.8613  0.8698

In [47]:
display(data.shape)
data.Target.value_counts()

(19538, 26)

Target
0    17685
1     1403
2      450
Name: count, dtype: int64

In [48]:
data

age  on_thyroxine  query_on_thyroxine  on_antithyroid_medication  \
0      79.0             0                   0                          0   
1      50.0             0                   0                          1   
2      63.0             0                   0                          0   
3      58.0             0                   0                          0   
4      39.0             0                   0                          0   
...     ...           ...                 ...                        ...   
19781  59.0             0                   0                          0   
19782  51.0             0                   0                          0   
19783  51.0             0                   0                          0   
19784  35.0             0                   0                          0   
19785  73.0             0                   0                          0   

       sick  pregnant  thyroid_surgery  I131_treatment  query_hypothyroid  \
0         0         0                0             0.0                  0   
1         0         0                0             0.0                  0   
2         0         0                0             0.0                  0   
3         0         0                0             0.0                  1   
4         0         0                0             0.0                  0   
...     ...       ...              ...             ...                ...   
19781     0         0                0             0.0                  0   
19782     0         0                0             0.0                  0   
19783     0         0                0             0.0                  0   
19784     0         0                0             0.0                  0   
19785     0         0                0             0.0                  0   

       query_hyperthyroid  ...     TSH  T3_measured    T3  TT4_measured  \
0                       0  ...   0.005            1  3.40             1   
1                       1  ...   0.030            1  3.40             1   
2                       0  ...     NaN            0   NaN             1   
3                       1  ...   0.200            0   NaN             1   
4                       1  ...   0.020            1  4.00             1   
...                   ...  ...     ...          ...   ...           ...   
19781                   0  ...   0.250            1  2.08             1   
19782                   0  ...  10.600            1  0.60             1   
19783                   0  ...   0.076            1  2.01             1   
19784                   0  ...   0.280            1  2.01             1   
19785                   0  ...   0.056            1  2.01             1   

         TT4  T4U_measured   T4U  FTI_measured     FTI  Target  
0      151.0             1  0.74             1  203.00       2  
1      131.0             1  1.02             1  129.00       2  
2      162.0             1  0.81             1  200.00       2  
3      256.0             1  0.90             1  283.00       2  
4      209.0             1  1.06             1  197.00       2  
...      ...           ...   ...           ...     ...     ...  
19781    7.9             1  9.90             1    8.00       0  
19782    0.5             1  8.90             1    0.55       2  
19783    9.0             1  6.70             1   13.40       0  
19784    9.0             1  8.90             1   10.10       0  
19785    8.1             1  9.00             1    9.00       0  

[19538 rows x 26 columns]

In [49]:
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import pandas as pd

# Load your trained XGBoost model
# Dummy code for demonstration purposes (replace with actual model loading)
model = XGBClassifier()

def predict_thyroid_condition():
    # Get user input
    t3 = float(input("Enter T3 value: "))
    t4 = float(input("Enter T4 value: "))
    tsh = float(input("Enter TSH value: "))
    age = float(input("Enter age: "))
    
    user_data = pd.DataFrame({
        'T3': [t3],
        'T4': [t4],
        'TSH': [tsh],
        'age': [age]
    })

    # Load your pre-fitted scaler (or fit it to your training data before using it)
    # Dummy code for demonstration purposes (replace with actual scaler loading or fitting)
    scaler = StandardScaler()
    # Fit the scaler to your training data before transforming new data
    # scaler.fit(training_data)
    
    user_data_scaled = scaler.transform(user_data)

    prediction = model.predict(user_data_scaled)[0]

    if prediction == 1:  # Assuming 1 represents hyperthyroid (adjust based on your model's class labels)
        print("Predicted Thyroid Condition: Hyperthyroid")
    elif prediction == 0:  # Assuming 0 represents hypothyroid (adjust based on your model's class labels)
        print("Predicted Thyroid Condition: Hypothyroid")
    else:
        print("Predicted Thyroid Condition: No Thyroid Condition")

# Call the function to get user input and make predictions
predict_thyroid_condition()


Enter T3 value:  0
Enter T4 value:  0
Enter TSH value:  0
Enter age:  0


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

X = data[['T3', 'TT4', 'TSH', 'age']]
y = data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = XGBClassifier()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

def predict_thyroid_condition():
    # Get user input
    t3 = float(input("Enter T3 value: "))
    t4 = float(input("Enter TT4 value: "))
    tsh = float(input("Enter TSH value: "))
    age = float(input("Enter age: "))
    
    user_data = pd.DataFrame({
        'T3': [t3],
        'T4': [t4],
        'TSH': [tsh],
        'age': [age]
    })

    user_data_scaled = scaler.transform(user_data)

    user_prediction = model.predict(user_data_scaled)

    # Display the predicted thyroid condition
    print("Predicted Thyroid Condition:", user_prediction[0])

# Call the function to get user input and make predictions
predict_thyroid_condition()


Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97      3516
           1       0.74      0.80      0.77       300
           2       0.71      0.71      0.71        92

    accuracy                           0.95      3908
   macro avg       0.81      0.82      0.82      3908
weighted avg       0.95      0.95      0.95      3908

Confusion Matrix:
 [[3407   83   26]
 [  61  239    0]
 [  25    2   65]]


Enter T3 value:  0
Enter TT4 value:  0
Enter TSH value:  0
Enter age:  0


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- T4
Feature names seen at fit time, yet now missing:
- TT4


In [52]:
def make_prediction(dataframe):
    # Preprocess data
    x = dataframe[corr_values.index]
    y = dataframe['Target']
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42) 
    
    # Make prediction
    X_train, X_test, y_train, y_test = holdout(dataframe)
    result = classification(classifiers, X_train, X_test, y_train, y_test)
    
    # Print output
    print("The predicted patient outcome is: ", result.index[0])
    print("The model's performance is as follows: ")
    display(result)

# Example usage
make_prediction(data)

Confusion matrix for:  XGBClassifier


array([[5166,   86,   27],
       [  49,  380,    4],
       [  37,    0,  113]], dtype=int64)

The predicted patient outcome is:  0.8651
The model's performance is as follows: 


Classifier  Accuracy  Precision  Recall
FScore                                            
0.8651  XGBClassifier    0.9654     0.8613  0.8698

In [ ]:
def make_prediction():
    # Get user input
    data = input("Please enter the patient data (separated by commas): ")
    
    # Split the input string into a list of strings
    data_list = data.split(',')
    
    # Convert the list of strings into a list of numbers
    data_numbers = [float(i) for i in data_list]
    
    # Convert the list of numbers into a DataFrame
    data = pd.DataFrame(data_numbers, columns=['Value'])
    
    # Additional steps: preprocess data, make prediction, and print output
    # ...

# Example usage
make_prediction()

In [ ]:
classifiers = {
    "XGBClassifier": XGBClassifier(learning_rate=0.01)
    # Add more classifiers if needed
}

# Load your data or use an existing DataFrame
# For simplicity, let's assume you have a DataFrame named 'data'

# Perform train-test split
X = data.drop("Target", axis=1)  # Assuming "Target" is the target variable
y = data["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Call the classification function
result = classification(classifiers, X_train, X_test, y_train, y_test)
display(result)

# Now, let's take input details from the user for a new testing set
# You need to modify this part based on your actual feature names
input_data = {}
for column in X.columns:
    value = input(f"Enter value for {column}: ")
    input_data[column] = [float(value)]

# Create a DataFrame with user input
user_input_df = pd.DataFrame(input_data)

# Use the trained model to make predictions on the user input
user_predictions = classifiers["XGBClassifier"].predict(user_input_df)

# Display the user input and the predicted class
print("\nUser Input:")
display(user_input_df)
print("\nPredicted Class:", user_predictions[0])